In [13]:
import os
os.chdir('..')

import json
from lightning_module import PreTrainLightning
from models import TSMVAE

from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from dataset import create_dataloaders

In [14]:
observational_data = json.load(open('/home/jp4474/latent-abc-smc/notebooks_lotka/lotka_volterra_data.json', 'r'))['y']

In [15]:
seq_len = 8 # TIME STAMPS
in_chans = 2 # Dimensions of time series

embed_dim = 64 # Embedding dimension
latent_dim = 32 # Latent dimension
num_heads = 4 # Number of attention heads
num_layers = 2 # Number of transformer layers
dropout = 0.1 # Dropout rate
depth = 2 # Depth of the model

decoder_embed_dim = latent_dim # Decoder embedding dimension
decoder_num_heads = 4 # Number of attention heads in the decoder
decoder_depth = 2 # Depth of the decoder
z_type = 'vae'
beta = 0.0001
noise_factor = 0.0

mask_ratio = 0.15
LR = 1e-4

In [16]:
model = TSMVAE(
            seq_len=seq_len, 
            in_chans=in_chans, 
            embed_dim=embed_dim, 
            num_heads=num_heads, 
            depth=depth,
            decoder_embed_dim=latent_dim, 
            decoder_num_heads=decoder_num_heads,
            decoder_depth=decoder_depth, 
            z_type=z_type, 
            lambda_=beta, 
            mask_ratio=mask_ratio,
            dropout=dropout,
            noise_factor=noise_factor,
        )

In [17]:
dirpath = '/home/jp4474/latent-abc-smc/tutorial'

In [18]:
lightning_module = PreTrainLightning(model=model, lr=LR, prog_bar=True)
checkpoint_callback = ModelCheckpoint(
            dirpath=dirpath,
            filename='TSMVAE-{epoch:02d}-{val_loss:.4f}',
            save_top_k=10,
            monitor='val_loss',
            mode='min'
        )

lr_monitor = LearningRateMonitor(logging_interval='epoch')
early_stop_callback = EarlyStopping(monitor="val_loss", patience=10, mode="min")

In [19]:
trainer = Trainer(
    max_epochs=100,
    accelerator='auto',
    devices=1,
    callbacks=[checkpoint_callback, lr_monitor, early_stop_callback],
    log_every_n_steps=10,
    enable_progress_bar=False,
    precision="32-true",
    fast_dev_run=False,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [20]:
train_dataloader, val_dataloader = create_dataloaders(
    data_dir='/home/jp4474/latent-abc-smc/data/Lotka',
    batch_size=32
)

In [21]:
trainer.fit(lightning_module, train_dataloader, val_dataloader)

/home/jp4474/latent-abc-smc/penv/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/jp4474/latent-abc-smc/tutorial exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params | Mode 
-----------------------------------------
0 | model | TSMVAE | 131 K  | train
-----------------------------------------
130 K     Trainable params
864       Non-trainable params
131 K     Total params
0.524     Total estimated model params size (MB)
97        Modules in train mode
0         Modules in eval mode

Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined